# Ассимптотика алгоритмов и зачем она нужна

Нужно как-то уметь сравнивать алгоритмы между собой, особенно хочется понимать какой из алгоритмов работает быстрее (вне зависимости от железа) И какой из них потребляет больеше памяти.
Для этого придумали О-нотацию.

Есть математическое определение: для двух функций f(x) и g(x) можно сказать что f является O(g) если существует такая константа C > 0, что при устремлении x к бесконечности (или  какой-то точке $x_0$) выполняется неравенство $|f(x)| < C * |g(x)|$

Если говорить человеческим языком, то если мы говорим что алгоритм работает за O(f(n)), где n - размер входных параметров алгоритма
это значит что скорость работы алгоритма с увеличением n будет расти не быстрее чем f(n) на какую-то константу

Вычислять ассимптотику можно просто отбрасывая все константы при вычислении количества операций, которое делате алгоритм, но некоторые случаи сложнее чем другие

Вот тут можно почитать еще https://bit.ly/3khIrsy

### Пример алгоритма с ассимптотикой (по времени) O($n^2$)

In [1]:
def n_square_algo(array):
    # Здесь n это len(array)
    
    # Этот цикл работает за O(n^2) операций
    for elem in array:
        for elem2 in array:
            print(elem * elem2)
    
    # Не влияет на ассимптотику т.к. отбрасываем все константы (не зависящие от n штуки)
    for i in range(100000000000000000):
        print(i)
    
    # Можно поверить что сортировка списка работает за O(n log n), что меньше чем O(n^2)
    array.sort()
    

# Задача 1 (В которой мы узнаем про трюк с префиксными суммами и что можно обменивать время на память)

Вам дан список чисел (как отрицательных так и положительных), задача найти в нем помассив с максимальной суммой элементов и при том из всех таких - с наибольшей длиной

Пример:

array = [1, 2, -4, 5, 2, -1, 3, -10, 7, 1, -1, 2]

ОТВЕТ: [1, 2, -4, <font color='green'>5, 2, -1, 3,</font> -10, 7, 1, -1, 1]

## Наивное решение с ассимптотикой O($n^3$) по времени и O($1$) по памяти:

In [2]:
def max_sum_subarray(array):
    n = len(array)
    ans = -1
    max_sum = float("-inf")
    
    for start in range(n):
        for finish in range(start + 1, n):
            current_sum = 0
            for elem in array[start:finish + 1]:
                current_sum += elem
                
            if current_sum > max_sum:
                max_sum, ans = current_sum, finish - start + 1
                
    return ans, max_sum


In [3]:
import numpy as np

In [4]:
%%timeit
max_sum_subarray(np.random.rand(100)-0.5)

55.9 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Чуть более близкое к оптимальному  решение с ассимптотикой O($n^2$) по времени и O($n$) по памяти:

In [5]:
def max_sum_subarray(array):
    # Трюк заключается в том чтобы сначала посчитать суммы всех префиксов массива и потом вычислять сумму любого подмассива за О(1)
    n = len(array)
    prefix_sum = [0]
    for i in range(n):
        prefix_sum.append(prefix_sum[-1] + array[i])
        
    ans = -1
    max_sum = -float("inf")
    for start in range(n):
        for finish in range(start + 1, n):
            # Вот как мы вычисляем сумму на подмассиве используя префиксные суммы
            current_sum = prefix_sum[finish + 1] - prefix_sum[start]
            if current_sum > max_sum:
                max_sum, ans = current_sum, finish - start + 1
                
    return ans, max_sum


In [6]:
%%timeit
max_sum_subarray(np.random.rand(1000)-0.5)

153 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
49.6/1.32

37.57575757575758

в 38 раз быстрее

# Задача 2 (Где мы узнаем про трюк с двумя указателями и что ассимптотику по времени можно сократить если итерироваться в каком-то специальном порядке):
Вам дано 2 массива с числами, отсортированные по возрастанию, задача найти по элементу в каждом изз массивов ($x_1$ и $x_2$ соответственно) таких что $|x_1 - x_2|$ минимально

Пример:

<font color='green'>array1</font> = [-10, -3, 0, 5, 13, 58, 91, 200, 356, 1000, 25000]

<font color='red'>array2</font> = [-9034, -574, -300, -29, 27, 100, 250, 340, 900, 60000]

ОТВЕТ: <font color='green'>91</font> и <font color='red'>100</font>

# Наивное решение с ассимптотикой O($n * m$) по времени и O(1) по памяти
где n и m это размеры массивов

In [8]:
def min_difference_pair(arr1, arr2):
    n, m = len(arr1) - 1, len(arr2) - 1
    min_diff, ans = abs(arr1[0] - arr2[0]), (0, 0)
    
    for i in range(n):
        for j in range(m):
            diff = abs(arr1[i] - arr2[j])
            if diff < min_diff:
                min_diff, ans = diff, (i, j)
            
    return min_diff, ans

# Оптимальное решение с ассимптотикой O($n + m$) по времени и O(1) по памяти

In [9]:
def min_difference_pair(arr1, arr2):
    n, m = len(arr1) - 1, len(arr2) - 1
    pointer1, pointer2 = 0, 0
    
    min_diff, ans = abs(arr1[0] - arr2[0]), (0, 0)
    
    while pointer1 + pointer2 != n + m:
        if pointer2 == m or (pointer1 < n and arr1[pointer1] <= arr2[pointer2]):
            pointer1 += 1
        
        elif pointer1 == n or (pointer2 < m and arr1[pointer1] >= arr2[pointer2]):
            pointer2 += 1
        
        current_diff = abs(arr1[pointer1] - arr2[pointer2])
        if current_diff < min_diff:
            min_diff, ans = current_diff, (pointer1, pointer2)
            
    return min_diff, ans
    

# Домашнее Задание

1) Потренироваться в рекурсии, например, здесь: https://informatics.mccme.ru/mod/statements/view.php?id=2543#1 (задачи в менюшке справа, нужно зарегаться чтобы решать)

2) Задачи на метод двух указателей:

#### Простые:

https://leetcode.com/problems/longest-substring-without-repeating-characters/
https://leetcode.com/problems/remove-duplicates-from-sorted-array/
https://leetcode.com/problems/merge-sorted-array/

#### Посложнее: 

https://leetcode.com/problems/long-pressed-name/
https://leetcode.com/problems/trapping-rain-water/

3) Придумать решение первой задачи с ассимптотикой по времени O(n) (например используя метод двух указателей)


In [10]:
def max_sum_subarray(array):
    """выдает длину и сумму подстроки, которая дает максимальную сумму
    на массиве np.random.rand(1000)-0.5) работает в 37 раз быстрее,
    чем алогиртм с лекции """

    n = len(array)
    # all prefix sum compute
    prefix_sum = [0]
    for i in range(n):
        prefix_sum.append(prefix_sum[-1] + array[i])
    start, finish, start_last, finish_last = 0, 0, -1, -1
    start_old = -1
    finish_old = -1

    ans = -1
    max_sum = -float("inf")
    while start != n - 1:
        # критерий остановки - дошли до предпоследнего места указателем начала
        if finish == n or ((finish == finish_old) and (start == start_old)):
            # если дошли указателем конца или два раза смотрим то же место - двигать начало
            start += 1
            current_sum = prefix_sum[finish + 1] - prefix_sum[start]
            if current_sum > max_sum:
                max_sum, ans = current_sum, finish - start + 1

        try:
            # если доходим до конца, следующего движения уже нет, поэтому, следующая сумма даст IndexError,
            # переставляю индикатор движения вперед по более большой сумме на False
            sum_check = (prefix_sum[finish] - prefix_sum[start + 1]) < (prefix_sum[finish + 2] - prefix_sum[start])
        except IndexError:
            sum_check = False
        if ((finish - start) < 1 or sum_check) and (finish + 2 < len(prefix_sum)):
            finish += 1
            current_sum = prefix_sum[finish + 1] - prefix_sum[start]
            if current_sum > max_sum:
                max_sum, ans = current_sum, finish - start + 1
        start_old = start_last
        start_last = start
        finish_old = finish_last
        finish_last = finish

    return ans, max_sum


In [11]:
not(True)

False

In [12]:
%%timeit
X = [1, 2, -4, 5, 2, -1, 3, -10, 7, 1, -1, 2]

max_sum_subarray(X)

30.2 µs ± 6.33 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
%%timeit
max_sum_subarray(np.random.rand(1000)-0.5)

3.64 ms ± 653 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


хотим быстрее 1 мс

In [14]:
X = np.random.rand(1000)-0.5

In [15]:
133/3.54

37.570621468926554

In [16]:
max_sum_subarray(X)

(35, 5.1941988669002)

In [17]:
max_sum_subarray(X)

(35, 5.1941988669002)